In [ ]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages
    !pip install ase torch_geometric
    import torch
    from torch_geometric.data import Data, DataLoader

    dataset = "/content/drive/My Drive"
else:
    dataset = "Dataset"

In [42]:
import pandas as pd
import numpy as np
import scipy

In [ ]:
path_pkl = f"{dataset}/absorption_mp_data.pkl"
pkl_df = pd.read_pickle(path_pkl)
pkl_df.head()

,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
0,mp-546266,DyBi2IO4,"(Atom('Dy', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0277, 0.0554, 0.0831, 0.1109, 0.1386, ...","[0.0, 0.8362304735329499, 3.309178082718349, 7...","[0.0, 0.0015666666666666665, 0.0031, 0.0046666...","[6.9163, 6.916533333333334, 6.9172, 6.9182, 6....",55.4328,1.3818000000000001
1,mp-9583,K2ZnF4,"(Atom('K', [np.float64(1.2531985199117757), np...","[0.0, 0.0399, 0.0798, 0.1197, 0.1596, 0.1995, ...","[0.0, 0.1362578822994739, 0.4541860898852432, ...","[0.0, 0.0001, 0.00016666666666666666, 0.0002, ...","[2.2021333333333333, 2.202166666666667, 2.2022...",79.7804,4.3866
2,mp-22988,CsGeCl3,"(Atom('Cs', [np.float64(7.78333021211009), np....","[0.0, 0.029, 0.058, 0.087, 0.116, 0.1449, 0.17...","[0.0, 0.515935972729374, 2.0145311056971553, 4...","[0.0, 0.0007, 0.0013666666666666669, 0.0020333...","[3.975733333333333, 3.9758333333333336, 3.9761...",57.9759,2.178
3,mp-861502,AcFeO3,"(Atom('Ac', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0587, 0.1174, 0.176, 0.2347, 0.2934, 0...","[0.0, 11295.053197761259, 44864.495977063896, ...","[0.0, 30.1914, 49.9196, 37.98413333333333, 21....","[67.54320000000001, 59.6215, 29.6067, 1.281, -...",117.3625,0.0
4,mp-1025029,PrHSe,"(Atom('Pr', [np.float64(-2.0265362), np.float6...","[0.0, 0.0365, 0.073, 0.1095, 0.1459, 0.1824, 0...","[0.0, 1.6849892209331498, 6.697792903781345, 1...","[0.0, 0.0026999999999999997, 0.005366666666666...","[8.784633333333334, 8.7851, 8.786566666666667,...",72.9718,1.9455999999999998


In [4]:
sample_structure = np.array(pkl_df["structure"][0])
sample_structure

array([Atom('Dy', [np.float64(0.0), np.float64(0.0), np.float64(0.0)], index=0),
       Atom('Bi', [np.float64(1.972872), np.float64(1.972872), np.float64(2.4905380260529997)], index=1),
       Atom('Bi', [np.float64(1.972872), np.float64(1.972872), np.float64(7.216928973947)], index=2),
       Atom('I', [np.float64(0.0), np.float64(0.0), np.float64(4.8537335)], index=3),
       Atom('O', [np.float64(1.972872), np.float64(0.0), np.float64(8.327443783812999)], index=4),
       Atom('O', [np.float64(0.0), np.float64(1.972872), np.float64(8.327443783812999)], index=5),
       Atom('O', [np.float64(1.972872), np.float64(0.0), np.float64(1.380023216187)], index=6),
       Atom('O', [np.float64(0.0), np.float64(1.972872), np.float64(1.380023216187)], index=7)],
      dtype=object)

## Krammer-Kronig's relation

$\epsilon_1(\omega)$ = real part of the dielectric function

$\epsilon_2(\omega)$ = imaginary part of the dielectric funcion

$\alpha (\omega)$ = absorption coefficient

$n(\omega)$ = real part of the refractive index

$k(\omega)$ = imaginary part of the refractive index

$R(\omega)$ = reflectance

$\omega$ = angular frequency = $\frac{\text{energy}}{\hbar}$

In [53]:
print(scipy.constants.Planck)
print(scipy.constants.physical_constants["electron volt-joule relationship"][0])
print(scipy.constants.pi)
print(scipy.constants.c)


6.62607015e-34
1.602176634e-19
3.141592653589793
299792458.0


In [ ]:
import scipy


class Krammer_Kronig:
    def __init__(self, real_dielectric, imag_dielectric, photon_energy):
        self.real_dielectric = real_dielectric
        self.imag_dielectric = imag_dielectric
        self.photon_energy = photon_energy * scipy.constants.physical_constants["electron volt-joule relationship"][0]
        self.reduced_plank = scipy.constants.Planck/(2*scipy.constants.pi)
        self.ang_frequency = self.photon_energy/self.reduced_plank

    
    def get_absorption_coff(self):
        left= (np.sqrt(2)*self.ang_frequency)/(scipy.constants.c)
        first_square_root = np.sqrt(np.square(self.real_dielectric) + np.square(self.imag_dielectric))
        right = np.sqrt(first_square_root - self.real_dielectric)
        # right = np.sqrt(np.sqrt(np.square(self.real_dielectric) + np.square(self.imag_dielectric)) - self.real_dielectric)
        self.absorption_coff = left * right
        return self.absorption_coff

    # Since we can get n, k, and R from the real and iaginary parts of the dielectric...    
    def get_real_refractive_index(self):
        self.real_refractive = np.sqrt(0.5*(np.sqrt(np.square(self.real_dielectric)+np.square(self.imag_dielectric))+self.real_dielectric))
        return self.real_refractive

    def get_imag_refractiv_index(self):
        self.imag_refractive = np.sqrt(0.5*(np.sqrt(np.square(self.real_dielectric)+np.square(self.imag_dielectric))-self.real_dielectric))
        return self.imag_refractive

    def get_reflectance(self):
        num = np.square(self.real_refractive - 1) + np.squar(self.imag_refractive)
        den = np.square(self.real_refractive + 1) + np.squar(self.imag_refractive)
        self.reflectance = num/den
        return self.reflectance


In [55]:
# Get absorption coefficient
sample_real = pkl_df["real_dielectric"][0][2]
sample_imag = pkl_df["imag_dielectric"][0][2]
sample_energy = pkl_df["energies"][0][2]

print(sample_real)
print(sample_imag)
print(sample_energy)

inst_1 = Krammer_Kronig(sample_real,sample_imag,sample_energy)
result_coff = inst_1.get_absorption_coff()

the_ref = pkl_df["absorption_coefficient"][0][2]

if result_coff == the_ref:
    print("calculations work perfectly")
else:
    print(f"Actual value: {the_ref}\nResult value: {result_coff}")



6.9172
0.0031
0.0554
Actual value: 3.309178082718349
Result value: 330.917558161353


Why is the $\text{result value} = \text{actual value} *100$?